In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Load Datas

In [3]:
df = pd.read_csv('sdn_ml.csv')
df.index = df['timestamp']
df.drop('timestamp', axis=1, inplace=True)
df.head()

,hour,minut,second,day,month,delay,sensor_id,temperature,label
timestamp,,,,,,,,,
2004-03-31 02:58:45,2,58,45,31,3,26.0,3,122.153,1
2004-02-28 00:58:46,0,58,46,28,2,26.0,3,20.204,1
2004-03-30 16:04:46,16,4,46,30,3,26.0,3,122.153,1
2004-03-30 14:43:05,14,43,5,30,3,26.0,3,122.153,1
2004-03-30 16:06:06,16,6,6,30,3,26.0,3,122.153,1


# Separing dataset

In [4]:
train_size = int(len(df) * 0.9)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]


# Normalizing

In [5]:
from sklearn.preprocessing import RobustScaler

In [6]:
f_columns = ['delay','temperature']

f_transformer = RobustScaler()
lb_transformer = RobustScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:,f_columns] = f_transformer.transform(train[f_columns].to_numpy())

#lb_transformer = lb_transformer.fit(train[['label']])
#train['label'] = lb_transformer.transform(train[['label']])

C:\Users\silva\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [14]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i: (i+time_steps),5:8].to_numpy()
        Xs.append(v)
        ys.append([y.iloc[i+time_steps],X.iloc[i:i+time_steps,5]])
    return np.array(Xs), np.array(ys)

In [15]:
TIME_STEPS = 30

X_train,Y_train = create_dataset(train, train.label, time_steps=TIME_STEPS)
X_test,Y_test = create_dataset(test, test.label, time_steps=TIME_STEPS)

In [16]:
Y_train.shape

(41543, 2)

In [11]:
print(type(X_train), type(Y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


# Training

In [12]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units=128,
            input_shape=(X_train.shape[1],X_train.shape[2])
        )
    ))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1))
#model.add(tf.keras.layers.ThresholdedReLU(theta=1))

# Compilation

In [13]:
loss ="mse"
optim = tf.keras.optimizers.SGD(
    learning_rate=0.001)
metrics=["accuracy"]

In [14]:
model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [ ]:
history = model.fit(
    X_train, Y_train, 
    epochs=50, 
    batch_size= 32,
    validation_split=0.1,
    shuffle=False
)

Epoch 1/50
1169/1169 [==============================] - 57s 42ms/step - loss: 0.0177 - accuracy: 0.9959 - val_loss: 1.4890e-04 - val_accuracy: 1.0000
Epoch 2/50
1169/1169 [==============================] - 44s 38ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 9.3507e-05 - val_accuracy: 1.0000
Epoch 3/50
1169/1169 [==============================] - 45s 39ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 7.6407e-05 - val_accuracy: 1.0000
Epoch 4/50
1169/1169 [==============================] - 44s 38ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 6.8999e-05 - val_accuracy: 1.0000
Epoch 5/50
1169/1169 [==============================] - 42s 36ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 4.1459e-05 - val_accuracy: 1.0000
Epoch 6/50
1169/1169 [==============================] - 41s 35ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 5.5275e-05 - val_accuracy: 1.0000
Epoch 7/50
1169/1169 [==============================] - 42s 36ms/step - loss: 0.0033 - accuracy: 1.0

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend();

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend();

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.plot(Y_test, marker='.', label='true')
plt.plot(y_pred,'r',marker='.', label='predicted')
plt.legend();

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
#confusion_matrix(y_test_inv, y_pred_inv)

In [ ]:
y_pred

In [ ]:
Y_test

In [ ]:
# y_train_inv = lb_transformer.inverse_transform(Y_train.reshape(1,-1)) 
# y_test_inv = lb_transformer.inverse_transform(Y_test.reshape(1,-1))
# y_pred_inv = lb_transformer.inverse_transform(y_pred)